In [ ]:
!pip3 install transformers
!pip3 install torch



     |████████████████████████████████| 3.1 MB 12.8 MB/s 
     |████████████████████████████████| 895 kB 45.0 MB/s 
     |████████████████████████████████| 596 kB 40.9 MB/s 
     |████████████████████████████████| 3.3 MB 49.5 MB/s 
     |████████████████████████████████| 59 kB 6.8 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


In [ ]:
from transformers import pipeline, AutoTokenizer, AutoModelForSequenceClassification
import torch 
import torch.nn.functional as F

In [ ]:
test_url = 'https://raw.githubusercontent.com/data-students/datathon2021/main/Reviews_Challenge/test_reviews.json'
train_url = 'https://raw.githubusercontent.com/data-students/datathon2021/main/Reviews_Challenge/train_reviews.json'
import requests

def get_ratings_and_descriptions_from_url(url):
    raw_data = requests.get(url).json()
    ratings = []
    descriptions = []
    for restaurant in raw_data:
        for review in restaurant.get('reviews_data'):
            rating = review.get('rating')
            if isinstance(rating, float):
                positive = 0
                if rating >= 3.5: positive = 1
                ratings += [positive]
                descriptions += [review.get('snippet')]
    return ratings, descriptions


train_labels, train_texts = get_ratings_and_descriptions_from_url(train_url)
test_labels, test_texts = get_ratings_and_descriptions_from_url(test_url)

In [ ]:
! wget http://ai.stanford.edu/~amaas/data/sentiment/aclImdb_v1.tar.gz
! tar -xf aclImdb_v1.tar.gz

--2021-11-13 15:43:19--  http://ai.stanford.edu/~amaas/data/sentiment/aclImdb_v1.tar.gz
Resolving ai.stanford.edu (ai.stanford.edu)... 171.64.68.10
Connecting to ai.stanford.edu (ai.stanford.edu)|171.64.68.10|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 84125825 (80M) [application/x-gzip]
Saving to: ‘aclImdb_v1.tar.gz’

aclImdb_v1.tar.gz   100%[===================>]  80.23M  19.2MB/s    in 5.8s    

2021-11-13 15:43:25 (14.0 MB/s) - ‘aclImdb_v1.tar.gz’ saved [84125825/84125825]



In [ ]:
from sklearn.model_selection import train_test_split
train_texts, val_texts, train_labels, val_labels = train_test_split(train_texts, train_labels, test_size=.2)

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
from transformers import pipeline, AutoTokenizer, AutoModelForSequenceClassification
import torch 
import torch.nn.functional as F
test_url = 'https://raw.githubusercontent.com/data-students/datathon2021/main/Reviews_Challenge/test_reviews.json'
train_url = 'https://raw.githubusercontent.com/data-students/datathon2021/main/Reviews_Challenge/train_reviews.json'
import requests
from sklearn.model_selection import train_test_split
from transformers import DistilBertTokenizerFast
import torch
from transformers import DistilBertForSequenceClassification, Trainer, TrainingArguments

#Gets the data
def get_ratings_and_descriptions_from_url(url):
    raw_data = requests.get(url).json()
    ratings = []
    descriptions = []
    for restaurant in raw_data:
        for review in restaurant.get('reviews_data'):
            rating = review.get('rating')
            if isinstance(rating, float):
                positive = 0
                if rating >= 3.5: positive = 1
                ratings += [positive]
                descriptions += [review.get('snippet')]
    return ratings, descriptions


train_labels, train_texts = get_ratings_and_descriptions_from_url(train_url)
test_labels, test_texts = get_ratings_and_descriptions_from_url(test_url)

train_texts, val_texts, train_labels, val_labels = train_test_split(train_texts, train_labels, test_size=.2)

model_name= "distilbert-base-uncased-finetuned-sst-2-english"
model = DistilBertForSequenceClassification.from_pretrained(model_name)
tokenizer = DistilBertTokenizerFast.from_pretrained(model_name)
#encode the text into tokens
#tokenizer = DistilBertTokenizerFast.from_pretrained('distilbert-base-uncased')
train_encodings = tokenizer(train_texts, truncation=True, padding=True)
val_encodings = tokenizer(val_texts, truncation=True, padding=True)
test_encodings = tokenizer(test_texts, truncation=True, padding=True)



class IMDbDataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels[idx])
        return item

    def __len__(self):
        return len(self.labels)

train_dataset = IMDbDataset(train_encodings, train_labels)
val_dataset = IMDbDataset(val_encodings, val_labels)
test_dataset = IMDbDataset(test_encodings, test_labels)



training_args = TrainingArguments(
    output_dir='./results',          # output directory
    num_train_epochs=1,              # total number of training epochs
    per_device_train_batch_size=16,  # batch size per device during training
    per_device_eval_batch_size=64,   # batch size for evaluation
    warmup_steps=500,                # number of warmup steps for learning rate scheduler
    weight_decay=0.01,               # strength of weight decay
    logging_dir='./logs',            # directory for storing logs
    logging_steps=10,
)

#model = DistilBertForSequenceClassification.from_pretrained("distilbert-base-uncased")

trainer = Trainer(
    model=model,                         # the instantiated 🤗 Transformers model to be trained
    args=training_args,                  # training arguments, defined above
    train_dataset=train_dataset,         # training dataset
    eval_dataset=val_dataset             # evaluation dataset
)

trainer.train()
torch.save(model, './trained_model')

Downloading:   0%|          | 0.00/629 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/255M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/226k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

***** Running training *****
  Num examples = 4
  Num Epochs = 1
  Instantaneous batch size per device = 16
  Total train batch size (w. parallel, distributed & accumulation) = 16
  Gradient Accumulation steps = 1
  Total optimization steps = 1


Step,Training Loss




Training completed. Do not forget to share your model on huggingface.co/models =)


